# Sprint14 Keras

## Keras とは
>Keras は元々はTheano、TensorFLowなど複数のフレームワークを扱いやすくする ラッパー として登場したライブラリでしたが、後にTensorFLowの 高レベルAPI として使われるようになりました。
TensorFLowに含まれる形のKerasであるtf.kerasを主に使っていきます。

>《ラッパーとは》
ラッパーはもともとのプログラムの機能を利用して、より使いやすいものを提供します。TensorFlowはニューラルネットワークに必要な計算を効率的に行う機能を提供しますが、初期のころはモデルを構築して学習を行うとなると手間がかかる部分もありました。そのため、TensorFlowをラップして、扱いやすくするKerasが登場しました。

>《高レベルAPIとは》
大きな単位で機能を簡単に扱えるように作られたものが高レベルAPIです。対義語として、細かい単位で機能をいじれるが、扱いがその分大変な低レベルAPIがあります。
TensorFlow自体でもニューラルネットワークのモデル構築や学習を行いやすくするために、高レベルAPIの充実が進められています。tf.Kerasはそのひとつです。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from numpy import linalg as LA
sns.set()
%matplotlib inline
import time
import copy

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import tensorflow as tf
from keras import backend as K


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

### ロジスティック回帰の実装

In [3]:
# ANDゲートの学習データを用意
x_train = np.array([[0,0],[0,1],[1,0],[1,1]]) #(4, 2)
y_train = np.array([[0],[0],[0],[1]]) #(4, 1)

## 複数の記述方法
>Kerasでは簡素にニューラルネットワークが記述できます。その書き方にはSequentialモデルとFunctional APIの2種類があります。

## Sequentialモデル
>Sequentialクラスを使用した記述方法です。
tf.keras.models.Sequential | TensorFlow

#### 層のインスタンスをSequentialクラスのコンストラクタにリストで渡すことでモデルを定義します。層のクラスについては以下のページにまとまっています。
Module: tf.keras.layers | TensorFlow

ロジスティック回帰を作るために、全結合層のクラス、tf.keras.layers.Denseを使います。引数に出力のユニット数、活性化関数、入力のユニット数を入れます。

In [4]:
model = tf.keras.Sequential([tf.keras.layers.Dense(1, 
                                                   activation = tf.nn.sigmoid, 
                                                   input_shape=(2,))])
#引数に出力のユニット数、活性化関数、入力のユニット数を入れます。


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


#### Denseクラスは引数で重みの初期化方法、バイアスの有無などの指定も可能です。

tf.keras.layers.Dense | TensorFlow

作成したモデルの構造はsummaryメソッドで確認することができます。層ごとの出力のshapeとパラメータ数が併記されます。

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


#### 構造が記述できたら、モデルをコンパイルします。
>コンパイル時に損失関数と最適化手法、評価関数を指定します。損失関数は名前をstringで指定します。ここでは2値分類のため、binary_crossentropyとなります。多値分類の場合はcategorical_crossentropy、回帰の場合はmean_squared_errorのようになります。

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


>そして学習を行います。scikit-learn同様にfitメソッドを使う設計になっています。verboseは学習過程の可視化方法のパラメータで、デフォルトの1ではバッチごとに更新されるプログレスバーが表示されます。verboseが0の場合は表示を行わず、2の場合はエポック毎の表示になります。

In [7]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

Epoch 1/1000
4/4 [==============================] - 0s 86ms/sample - loss: 0.7698 - acc: 0.5000
Epoch 2/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7627 - acc: 0.5000
Epoch 3/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7569 - acc: 0.5000
Epoch 4/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7543 - acc: 0.5000
Epoch 5/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7502 - acc: 0.5000
Epoch 6/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7463 - acc: 0.5000
Epoch 7/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7438 - acc: 0.5000
Epoch 8/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7390 - acc: 0.5000
Epoch 9/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.7356 - acc: 0.5000
Epoch 10/1000
4/4 [==============================] - 0s 1ms/sample - loss: 0.7330 - acc: 0.5000
Epoch 11/1000
4/4 [=============================

>今は用意していませんが、検証用データがある場合は、引数validation_dataに与えることで、エポック毎の検証も可能です。



In [8]:
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1,
                    validation_data=(x_train, y_train))

Train on 4 samples, validate on 4 samples
Epoch 1/1000
4/4 [==============================] - 0s 49ms/sample - loss: 0.1010 - acc: 1.0000 - val_loss: 0.1008 - val_acc: 1.0000
Epoch 2/1000
4/4 [==============================] - 0s 4ms/sample - loss: 0.1009 - acc: 1.0000 - val_loss: 0.1006 - val_acc: 1.0000
Epoch 3/1000
4/4 [==============================] - 0s 3ms/sample - loss: 0.1007 - acc: 1.0000 - val_loss: 0.1005 - val_acc: 1.0000
Epoch 4/1000
4/4 [==============================] - 0s 3ms/sample - loss: 0.1005 - acc: 1.0000 - val_loss: 0.1004 - val_acc: 1.0000
Epoch 5/1000
4/4 [==============================] - 0s 3ms/sample - loss: 0.1004 - acc: 1.0000 - val_loss: 0.1002 - val_acc: 1.0000
Epoch 6/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.1003 - acc: 1.0000 - val_loss: 0.1001 - val_acc: 1.0000
Epoch 7/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.1003 - acc: 1.0000 - val_loss: 0.0999 - val_acc: 1.0000
Epoch 8/1000
4/4 [===============

#### 推定もscikit-learn同様にpredictメソッドを使います。



In [11]:
y_pred_proba = model.predict(x_train)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

y_pred_proba [0.0024536  0.10709944 0.10622472 0.852849  ]
y_pred [0 0 0 1]


#### 結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。



In [12]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.09680259227752686
Train accuracy: 1.0


### Sequentialモデルのもうひとつの書き方
>Sequentialモデルでは、コンストラクタで層のクラスを渡さず、addメソッドを使って記述する方法もよく使われます。



In [13]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1,
                                activation = tf.nn.sigmoid, 
                                input_shape=(2,)))

### 複数層の場合
>ロジスティック回帰ではなく、2層のニューラルネットワークの場合は以下のように記述できます。2層目以降はinput_shapeを与える必要がありません。tf.kerasが自動的に計算するためです。

In [14]:
model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, 
                                  activation = tf.nn.relu,
                                  input_shape=(2,)),
            tf.keras.layers.Dense(1, 
                                  activation = tf.nn.sigmoid)])

In [15]:
#addメソッドを使えば次のようになります。

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, 
                                activation = tf.nn.relu,
                                input_shape=(2,)))
model.add(tf.keras.layers.Dense(1,
                                activation = tf.nn.sigmoid))

## Functional API
>Functional APIを使えばより自由度の高いモデル構築が行えます。Sequentialクラスの代わりにModelクラスを使用します。
tf.keras.models.Model | TensorFlow

入力から出力までの流れを記述していき、最後にModelクラスに入力層と出力層のインスタンスを渡します。

In [16]:
input_data = tf.keras.layers.Input(shape=(2,)) # 入力層
output = tf.keras.layers.Dense(1, 
                               activation=tf.nn.sigmoid)(input_data) # 出力層

model = tf.keras.Model(inputs=input_data, outputs=output)

#### モデル構造の記述以降はSequentialモデルと全く同じです。

In [17]:
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Train on 4 samples
Epoch 1/1000
4/4 [==============================] - 0s 93ms/sample - loss: 0.8426 - acc: 0.7500
Epoch 2/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.8380 - acc: 0.7500
Epoch 3/1000
4/4 [==============================] - 0s 2ms/sample - loss: 0.8309 - acc: 0.7500
Epoch 4/1000
4/4 [==============================] - 0s 3ms/sample - loss: 0.8289 - acc: 0.7500
Epoch 5/1000
4/4 [==============================] - 0s 3ms/sample - loss: 0.8247 - acc: 0.7500
Epoch 6/1000
4/4 [================

#### 複数層の場合
>4層のニューラルネットワークは以下のように記述できます。



In [18]:
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=input_data, outputs=output)

#### この記述方法では枝分かれを表現することもできます。
以下は3層目で2つに枝分かれし、次の層で結合している例です。

tf.keras.layers.concatenate | TensorFlow

In [19]:
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
y1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
y2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
z = tf.keras.layers.concatenate([y1, y2])
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(z)
model = tf.keras.Model(inputs=input_data, outputs=output)

## ラッパーとしてのKeras
>ラッパーとしてのKerasもデフォルトでTensorFlowをバックエンドとして使用しているため、基本的な使い方は同じです。
ドキュメントが日本語でも公開されているため、tf.kerasを利用する上で参考にすることができます。例えば2つの記述方法については以下のページです。

Sequentialモデルのガイド - Keras Documentation
Functional APIのガイド - Keras Documentation

>compileメソッドで指定できる損失関数もまとまっています。

損失関数 - Keras Documentation

>Sequentialモデルは以下のように書けます。ロジスティック回帰の例です。
以下のコードのほとんどは上で紹介したtf.kerasと実質的に同じですが、例えば活性化関数を全結合層とは別のクラスとして渡しています。また、最適化手法の部分はtf.train.AdamOptimizerからkeras.optimizers.Adamに変わっています。tf.kerasではTensorFlow自体の最適化手法クラスを呼んでいるのに対し、KerasではKeras独自の最適化手法クラスを使用するためです。ラッパーとしてのKerasのコードも見る機会が多いですから若干の違いに慣れておくと良いでしょう。



In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(1, input_shape=(2,)))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=1000,
                    verbose=1)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 3         
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________





Epoch 1/1000





4/4 [==============================] - 1s 282ms/step - loss: 0.7752 - acc: 0.7500
Epoch 2/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.7696 - acc: 0.7500
Epoch 3/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.7662 - acc: 0.7500
Epoch 4/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.7631 - acc: 0.7500
Epoch 5/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.7599 - acc: 0.7500
Epoch 6/1000
4/4 [==============================] - 0s 2ms/step - l

## 【問題1】公式Exampleを分担して実行
>TensorFLowの公式Exampleを分担して実行してください。
以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。
- research
定番のモデルから最新のモデルまで多様なコードが公開されています。
models/research at master · tensorflow/models
- tutorials
TensorFLowのチュートリアルとして用意された簡単なモデルが含まれています。
models/tutorials at master · tensorflow/models

In [ ]:
"""
tutorials


Timing benchmark for AlexNet inference.
To run, use:
  bazel run -c opt --config=cuda \
      models/tutorials/image/alexnet:alexnet_benchmark
Across 100 steps on batch size = 128.
Forward pass:
Run on Tesla K40c: 145 +/- 1.5 ms / batch
Run on Titan X:     70 +/- 0.1 ms / batch
Forward-backward pass:
Run on Tesla K40c: 480 +/- 48 ms / batch
Run on Titan X:    244 +/- 30 ms / batch
"""

#コードを練習したくなったらtensorflow の research からデータをみる、
#信頼できるデータにアクセスすること。

#調べる人も聞く人も楽しい発表になるように。

## 異なるフレームワークへの書き換え
>「ディープラーニングフレームワーク1」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。
- Iris（Iris-versicolorとIris-virginicaのみの2値分類）
- Iris（3種類全ての目的変数を使用して多値分類）
- House Prices
- MNIST

### Kerasへの書き換え
>KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。
KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。



## 【問題2】Iris（2値分類）をKerasで学習


In [9]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [10]:
#Functional API

input_data = tf.keras.layers.Input(shape=(4,))
output = tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)(input_data)

model = tf.keras.Model(inputs = input_data, outputs = output)

In [11]:
model.summary()
model.compile(loss= 'binary_crossentropy',
              optimizer = tf.train.AdamOptimizer(learning_rate = 0.01),
              metrics = ['accuracy'])

story = model.fit(X_train, y_train,
                  batch_size = 1,
                  epochs = 100,
                  verbose = 1,
                  validation_data = (X_val, y_val))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
Train on 64 samples, validate on 16 samples
Epoch 1/100
64/64 [==============================] - 0s 7ms/sample - loss: 0.5652 - acc: 0.7656 - val_loss: 0.5843 - val_acc: 0.6875
Epoch 2/100
64/64 [==============================] - 0s 1ms/sample - loss: 0.5497 - acc: 0.7969 - val_loss: 0.5434 - val_acc: 0.8750
Epoch 3/100
64/64 [==============================] - 0s 1ms/sample - loss: 0.5264 - acc: 0.9062 - val_loss: 0.5445 - val_acc: 0.8750
Epoch 4/100
64/64 [==============================] - 0s 1ms/sample - loss: 0.5069 - ac

In [7]:
y_pred_proba = model.predict(X_test)[:,0]
y_pred = np.where(y_pred_proba > 0.5, 1, 0)

print("y_pred_proba",y_pred_proba)
print("y_pred",y_pred)

y_pred_proba [0.1940246  0.9804326  0.16739753 0.9851923  0.835789   0.98178804
 0.37475568 0.7993598  0.9855042  0.9475783  0.9318696  0.95154923
 0.98069596 0.28696442 0.04354948 0.08972093 0.6498308  0.06253028
 0.81346947 0.09418333]
y_pred [0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0]


In [12]:
score = model.evaluate(X_test, y_test, verbose = 0)
print(score)
print('Test loss', score[0])
print('Test accuracy', score[1])

[0.2331160604953766, 0.9]
Test loss 0.2331160604953766
Test accuracy 0.9


## 【問題3】Iris（多値分類）をKerasで学習


In [13]:
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", 
               "PetalLengthCm", "PetalWidthCm"]]
y = df["Species"]

y = np.array(y)
X = np.array(X) #(150, 4)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis] 

"""
one_hot化が必要
"""
onehot = OneHotEncoder(sparse=False)
y = onehot.fit_transform(y) #one_hot(150, 3)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#softmax
#多値分類

/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [14]:
#Functional API

K.clear_session() #clear_session

#引数に出力のユニット数、活性化関数、入力のユニット数を入れます。
input_data = tf.keras.layers.Input(shape = (4,))
output = tf.keras.layers.Dense(3, activation = tf.nn.softmax)(input_data)

model = tf.keras.Model(inputs = input_data, outputs = output)

model.summary()
model.compile(loss='categorical_crossentropy', #categorical_crossentropy
              optimizer = tf.train.AdamOptimizer(learning_rate = 0.01),
              metrics = ['accuracy'])




Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(X_train, y_train,
                    batch_size = 1,
                    epochs = 100,
                    verbose = 1,
                    validation_data = (X_val, y_val))

Train on 96 samples, validate on 24 samples
Epoch 1/100
96/96 [==============================] - 0s 3ms/sample - loss: 2.4524 - acc: 0.2083 - val_loss: 1.9797 - val_acc: 0.2917
Epoch 2/100
96/96 [==============================] - 0s 1ms/sample - loss: 1.6920 - acc: 0.3333 - val_loss: 1.3267 - val_acc: 0.3333
Epoch 3/100
96/96 [==============================] - 0s 989us/sample - loss: 1.1039 - acc: 0.3854 - val_loss: 0.8826 - val_acc: 0.6250
Epoch 4/100
96/96 [==============================] - 0s 980us/sample - loss: 0.7307 - acc: 0.7396 - val_loss: 0.6767 - val_acc: 0.7083
Epoch 5/100
96/96 [==============================] - 0s 966us/sample - loss: 0.6190 - acc: 0.7396 - val_loss: 0.5959 - val_acc: 0.7083
Epoch 6/100
96/96 [==============================] - 0s 1ms/sample - loss: 0.5451 - acc: 0.8021 - val_loss: 0.5292 - val_acc: 0.7083
Epoch 7/100
96/96 [==============================] - 0s 926us/sample - loss: 0.4902 - acc: 0.8229 - val_loss: 0.4934 - val_acc: 0.7083
Epoch 8/100
96/96

In [16]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)
print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

y_pred_proba [[5.3234039e-11 1.1539694e-03 9.9884605e-01]
 [2.4678709e-04 9.9257129e-01 7.1819210e-03]
 [9.9998283e-01 1.7185443e-05 2.6240235e-14]
 [1.4191471e-09 5.0919846e-02 9.4908011e-01]
 [9.9957591e-01 4.2403908e-04 1.0576109e-11]
 [2.3318058e-12 3.2995731e-04 9.9967003e-01]
 [9.9976712e-01 2.3288903e-04 5.3271007e-12]
 [4.8753107e-05 9.3906987e-01 6.0881298e-02]
 [2.1651862e-05 9.4052041e-01 5.9457973e-02]
 [4.5111054e-04 9.8562247e-01 1.3926525e-02]
 [9.3041599e-08 2.1543759e-01 7.8456235e-01]
 [1.1865359e-04 9.4659185e-01 5.3289518e-02]
 [5.2412841e-05 9.5017803e-01 4.9769446e-02]
 [2.2735559e-05 8.8602477e-01 1.1395248e-01]
 [2.6301483e-05 8.5898042e-01 1.4099327e-01]
 [9.9925739e-01 7.4261648e-04 2.0455193e-11]
 [3.1308984e-05 8.1805927e-01 1.8190944e-01]
 [5.5969791e-05 9.0038890e-01 9.9555239e-02]
 [9.9814427e-01 1.8557037e-03 2.1511153e-10]
 [9.9991858e-01 8.1378894e-05 7.5983631e-13]
 [9.9719566e-09 1.8934283e-02 9.8106569e-01]
 [2.9571573e-05 7.0619518e-01 2.9377523e-0

In [17]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.06062551960349083
Test accuracy: 1.0


## 【問題4】House PricesをKerasで学習


In [18]:
"""
初期化の順番が大事な時はinitialized_valueを使うこと。

歪度が大きいので、対数変換をして正規分布に近づけた方が良い。
"""
# データセットの読み込み
data = pd.read_csv('train.csv')
y = data['SalePrice'] #(1460,)
y = y.astype(np.int)[:, np.newaxis] #(1460, 1)
X = data.loc[:,['GrLivArea','YearBuilt']] #(1460, 2)
y = np.array(y).astype(np.float64)
X = np.array(X).astype(np.float64)

#対数変換
X = np.log(X)
y = np.log(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
#回帰
K.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(200, input_dim=2, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1))
model.compile(loss="mean_squared_error",optimizer = tf.train.AdamOptimizer(learning_rate=0.01),
             metrics=["mse"])
model.summary()

#Sequential

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 20,801
Trainable params: 20,801
Non-trainable params: 0
_________________________________________________________________


In [19]:
history=model.fit(X_train,y_train,batch_size=1,epochs=100,verbose=0)

In [21]:
y_pred=model.predict(X_test)
from sklearn.metrics import  mean_squared_error

print(np.sqrt(mean_squared_error(y_test, y_pred)))

0.4075243907544335


## 【問題5】MNISTをKerasで学習


In [33]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 67s 1ms/step - loss: 0.2640 - acc: 0.9185 - val_loss: 0.0548 - val_acc: 0.9818
Epoch 2/12
60000/60000 [==============================] - 69s 1ms/step - loss: 0.0883 - acc: 0.9735 - val_loss: 0.0440 - val_acc: 0.9851
Epoch 3/12
60000/60000 [==============================] - 69s 1ms/step - loss: 0.0664 - acc: 0.9800 - val_loss: 0.0370 - val_acc: 0.9875
Epoch 4/12
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0548 - acc: 0.9837 - val_loss: 0.0342 - val_acc: 0.9893
Epoch 5/12
60000/60000 [==============================] - 72s 1ms/step - loss: 0.0470 - acc: 0.9858 - val_loss: 0.0282 - val_acc: 0.9905
Epoch 6/12
60000/60000 [==============================] - 67s 1ms/step - 

## 【問題6】（アドバンス課題）PyTorchへの書き換え
>4種類の問題をPyTorchに書き換えてください。



## 【問題7】（アドバンス課題）フレームワークの比較
>それぞれのフレームワークにはどのような違いがあるかをまとめてください。
視点例
- 計算速度
- コードの行数・可読性
